In [37]:
# https://stackoverflow.com/questions/45350891/fitting-a-polynomial-using-np-polyfit-in-3-dimensions
# https://math.stackexchange.com/questions/1786495/estimating-the-curvature-of-a-discretized-curve-in-3d-with-cubic-splines

In [38]:
# %matplotlib qt
%load_ext autoreload
%autoreload 2
from utils.utils_IO import load_object, write_video_from_directory, write_video_from_list, read_frame_at_idx, plot_chin_spline
from utils.polyfitting import fit_3D_polynomial, \
        fit_polynomials_per_frame, \
        eval_3D_polynomials, \
            eval_polynomials_per_frame, compute_curvature_per_frame, compute_curvature_function 
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import interpolate
import cv2
from pathlib import Path
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from tqdm import tqdm
import re
from typeguard import typechecked
from typing import List, Callable

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
data_dict = load_object('/Volumes/sawtell-locker/C1/free/3d_reconstruction/V2/data_dict')

In [40]:
trial_number = 0
example_trial = data_dict["trials"]["Joao"]["pre"][trial_number]
video_path = Path(data_dict["video_paths"]["Joao"]["pre"]) / "concatenated_tracking.avi"
#reproj_path = Path(f'/Volumes/sawtell-locker/C1/free/3d_reconstruction/V2/{video_path.parent.stem}') / "reprojections.mov"
#if not Path.is_file(reproj_path):
#    raise ValueError(f"No reprojections for {video_path.parent.stem}")

In [41]:
# Collect columns with chin 
# chin_base, chin_1_4, chin_mid, chin_3_4, chin_end
chin_trial_df = example_trial.filter(regex='chin')
chin_trial_arr = np.reshape(chin_trial_df.values, (chin_trial_df.values.shape[0], -1, 3))
# Get bodypart names
bp_names = []
for col in chin_trial_df.columns:
    if col[0] not in bp_names:
        bp_names.append(col[0])

In [42]:
# fit polynomial to a sequence of time pts
coeffs_all_frames = fit_polynomials_per_frame(data_arr = chin_trial_arr, degree = 2)

100%|██████████| 1098/1098 [00:00<00:00, 2280.26it/s]


In [43]:
interpolation_points = np.linspace(0,4,20)

In [44]:
evals = eval_polynomials_per_frame(coeffs_all_frames, interpolation_points)

100%|██████████| 1098/1098 [00:00<00:00, 8182.90it/s]


In [45]:
from utils.polyfitting import compute_curvature_per_frame
curvatures = compute_curvature_per_frame(coeffs_all_frames=coeffs_all_frames, 
                                         interpolation_points=interpolation_points)

100%|██████████| 1098/1098 [00:00<00:00, 36132.55it/s]


In [46]:
frame_idx = 120
plt.subplot(141)
plt.plot(evals[frame_idx, 0, :])
plt.title('x')
plt.subplot(142)
plt.plot(evals[frame_idx, 1, :])
plt.title('y')
plt.subplot(143)
plt.plot(evals[frame_idx, 2, :])
plt.title('z')
plt.subplot(144)
plt.plot(curvatures[frame_idx, 5:])
plt.title('curvature')
plt.tight_layout()

In [61]:
from utils.utils_IO import plot_chin_spline
plot_chin_spline(chin_trial_arr=chin_trial_arr, bp_names=bp_names, 
                 poly_evals=evals, curvatures=curvatures, out_file="chin_movie.mov",
                 start_frame=500, end_frame=800)

100%|██████████| 300/300 [01:02<00:00,  4.78it/s]
